In [0]:
!pip install mlflow


Collecting mlflow
 Downloading mlflow-1.19.0-py3-none-any.whl (14.4 MB)
 | 10 kB 12.1 MB/s eta 0:00:02 | | 20 kB 14.4 MB/s eta 0:00:01 | | 30 kB 5.9 MB/s eta 0:00:03 | | 40 kB 6.8 MB/s eta 0:00:03 |▏ | 51 kB 5.2 MB/s eta 0:00:03 |▏ | 61 kB 5.9 MB/s eta 0:00:03 |▏ | 71 kB 6.4 MB/s eta 0:00:03 |▏ | 81 kB 7.1 MB/s eta 0:00:03 |▏ | 92 kB 6.4 MB/s eta 0:00:03 |▎ | 102 kB 5.2 MB/s eta 0:00:03 |▎ | 112 kB 5.2 MB/s eta 0:00:03 |▎ | 122 kB 5.2 MB/s eta 0:00:03 |▎ | 133 kB 5.2 MB/s eta 0:00:03 |▎ | 143 kB 5.2 MB/s eta 0:00:03 |▍ | 153 kB 5.2 MB/s eta 0:00:03 |▍ | 163 kB 5.2 MB/s eta 0:00:03 |▍ | 174 kB 5.2 MB/s eta 0:00:03 |▍ | 184 kB 5.2 MB/s eta 0:00:03 |▍ | 194 kB 5.2 MB/s eta 0:00:03 |▌ | 204 kB 5.2 MB/s eta 0:00:03 |▌ | 215 kB 5.2 MB/s eta 0:00:03 |▌ | 225 kB 5.2 MB/s eta 0:00:03 |▌ | 235 kB 5.2 MB/s eta 0:00:03 |▌ | 245 kB 5.2 MB/s eta 0:00:03 |▋ | 256 kB 5.2 MB/s eta 0:00:03 |▋ | 266 kB 5.2 MB/s eta 0:00:03 |▋ | 276 kB 5.2 MB/s eta 0:00:03 |▋ | 286 kB 5.2 MB/s eta 0:00:03 |▋ | 296 kB 5.2 MB/s eta 0:00:03 |▊ | 307 kB 5.2 MB/s eta 0:00:03 |▊ | 317 kB 5.2 MB/s eta 0:00:03 |▊ | 327 kB 5.2 MB/s eta 0:00:03 |▊ | 337 kB 5.2 MB/s eta 0:00:03 |▊ | 348 kB 5.2 MB/s eta 0:00:03 |▉ | 358 kB 5.2 MB/s eta 0:00:03 |▉ | 368 kB 5.2 MB/s eta 0:00:03 |▉ | 378 kB 5.2 MB/s eta 0:00:03 |▉ | 389 kB 5.2 MB/s eta 0:00:03 |█ | 399 kB 5.2 MB/s eta 0:00:03 |█ | 409 kB 5.2 MB/s eta 0:00:03 |█ | 419 kB 5.2 MB/s eta 0:00:03 |█ | 430 kB 5.2 MB/s eta 0:00:03 |█ | 440 kB 5.2 MB/s eta 0:00:03 |█ | 450 kB 5.2 MB/s eta 0:00:03 |█ | 460 kB 5.2 MB/s eta 0:00:03 |█ | 471 kB 5.2 MB/s eta 0:00:03 |█ | 481 kB 5.2 MB/s eta 0:00:03 |█ | 491 kB 5.2 MB/s eta 0:00:03 |█▏ | 501 kB 5.2 MB/s eta 0:00:03 |█▏ | 512 kB 5.2 MB/s eta 0:00:03 |█▏ | 522 kB 5.2 MB/s eta 0:00:03 |█▏ | 532 kB 5.2 MB/s eta 0:00:03 |█▏ | 542 kB 5.2 MB/s eta 0:00:03 |█▎ | 552 kB 5.2 MB/s eta 0:00:03 |█▎ | 563 kB 5.2 MB/s eta 0:00:03 |█▎ | 573 kB 5.2 MB/s eta 0:00:03 |█▎ | 583 kB 5.2 MB/s eta 0:00:03 |█▎ | 593 kB 5.2 MB/s eta 0:00:03 |█▍ | 604 kB 5.2 MB/s eta 0:00:03 |█▍ | 614 kB 5.2 MB/s eta 0:00:03 |█▍ | 624 kB 5.2 MB/s eta 0:00:03 |█▍ | 634 kB 5.2 MB/s eta 0:00:03 |█▍ | 645 kB 5.2 MB/s eta 0:00:03 |█▌ | 655 kB 5.2 MB/s eta 0:00:03 |█▌ | 665 kB 5.2 MB/s eta 0:00:03 |█▌ | 675 kB 5.2 MB/s eta 0:00:03 |█▌ | 686 kB 5.2 MB/s eta 0:00:03 |█▌ | 696 kB 5.2 MB/s eta 0:00:03 |█▋ | 706 kB 5.2 MB/s eta 0:00:03 |█▋ | 716 kB 5.2 MB/s eta 0:00:03 |█▋ | 727 kB 5.2 MB/s eta 0:00:03 |█▋ | 737 kB 5.2 MB/s eta 0:00:03 |█▋ | 747 kB 5.2 MB/s eta 0:00:03 |█▊ | 757 kB 5.2 MB/s eta 0:00:03 |█▊ | 768 kB 5.2 MB/s eta 0:00:03 |█▊ | 778 kB 5.2 MB/s eta 0:00:03 |█▊ | 788 kB 5.2 MB/s eta 0:00:03 |█▉ | 798 kB 5.2 MB/s eta 0:00:03 |█▉ | 808 kB 5.2 MB/s eta 0:00:03 |█▉ | 819 kB 5.2 MB/s eta 0:00:03 |█▉ | 829 kB 5.2 MB/s eta 0:00:03 |█▉ | 839 kB 5.2 MB/s eta 0:00:03 |██ | 849 kB 5.2 MB/s eta 0:00:03 |██ | 860 kB 5.2 MB/s eta 0:00:03 |██ | 870 kB 5.2 MB/s eta 0:00:03 |██ | 880 kB 5.2 MB/s eta 0:00:03 |██ | 890 kB 5.2 MB/s eta 0:00:03 |██ | 901 kB 5.2 MB/s eta 0:00:03 |██ | 911 kB 5.2 MB/s eta 0:00:03 |██ | 921 kB 5.2 MB/s eta 0:00:03 |██ | 931 kB 5.2 MB/s eta 0:00:03 |██ | 942 kB 5.2 MB/s eta 0:00:03 |██▏ | 952 kB 5.2 MB/s eta 0:00:03 |██▏ | 962 kB 5.2 MB/s eta 0:00:03 |██▏ | 972 kB 5.2 MB/s eta 0:00:03 |██▏ | 983 kB 5.2 MB/s eta 0:00:03 |██▏ | 993 kB 5.2 MB/s eta 0:00:03 |██▎ | 1.0 MB 5.2 MB/s eta 0:00:03 |██▎ | 1.0 MB 5.2 MB/s eta 0:00:03 |██▎ | 1.0 MB 5.2 MB/s eta 0:00:03 |██▎ | 1.0 MB 5.2 MB/s eta 0:00:03 |██▎ | 1.0 MB 5.2 MB/s eta 0:00:03 |██▍ | 1.1 MB 5.2 MB/s eta 0:00:03 |██▍ | 1.1 MB 5.2 MB/s eta 0:00:03 |██▍ | 1.1 MB 5.2 MB/s eta 0:00:03 |██▍ | 1.1 MB 5.2 MB/s eta 0:00:03 |██▍ | 1.1 MB 5.2 MB/s eta 0:00:03 |██▌ | 1.1 MB 5.2 MB/s eta 0:00:03 |██▌ | 1.1 MB 5.2 MB/s eta 0:00:03 |██▌ | 1.1 MB 5.2 MB/s eta 0:00:03 |██▌ | 1.1 MB 5.2 MB/s eta 0:00:03 |██▌ | 1.1 MB 5.2 MB/s eta 0:00:03 |██▋ | 1.2 MB 5.2 MB/s eta 0:00:03 |██▋ | 1.2

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import when
from pyspark.ml.feature import StringIndexer, IndexToString
import mlflow
import mlflow.spark
from mlflow.tracking import MlflowClient
from urllib.parse import urlparse

In [0]:
# Creating dataframe from Hive table
df = spark.sql("SELECT * FROM plays")
df.collect()

Out[2]: [Row(userid='d1391ee49ae39ddd48a9eb240d0dea3c733c20b2', artistid='d8c220a3-d9e9-4d8b-96c9-e0f58aa93310', artistname='flowing tears', plays='120'),
 Row(userid='d1391ee49ae39ddd48a9eb240d0dea3c733c20b2', artistid='9c615217-2827-46e0-b6a6-292eb232a950', artistname='rajna', plays='118'),
 Row(userid='d1391ee49ae39ddd48a9eb240d0dea3c733c20b2', artistid='078a9376-3c04-4280-b7d7-b20e158f345d', artistname='a perfect circle', plays='111'),
 Row(userid='d1391ee49ae39ddd48a9eb240d0dea3c733c20b2', artistid='a5eb1b55-1a67-4b2e-bf1b-ae575cf4a5c8', artistname='lykathea aflame', plays='110'),
 Row(userid='d1391ee49ae39ddd48a9eb240d0dea3c733c20b2', artistid='de1bbff3-602a-4c55-a232-117f088889df', artistname='lumsk', plays='109'),
 Row(userid='d1391ee49ae39ddd48a9eb240d0dea3c733c20b2', artistid='6e732849-3c75-4c5c-95b7-53f6e638fc0b', artistname="hagalaz' runedance", plays='104'),
 Row(userid='d1391ee49ae39ddd48a9eb240d0dea3c733c20b2', artistid='ec91c6e1-e63e-4461-b933-9cdf97d058ed', artistname='donna', plays='102'),
 Row(userid='d1391ee49ae39ddd48a9eb240d0dea3c733c20b2', artistid='49cd96a6-42c3-44f6-ba2a-cd9301046b96', artistname='burzum', plays='101'),
 Row(userid='d1391ee49ae39ddd48a9eb240d0dea3c733c20b2', artistid='db1b28f3-9dc5-4a07-ab0d-18c23e2b2cdf', artistname='forgotten woods', plays='96'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='a74b1b7f-71a5-4011-9441-d0b5e4122711', artistname='radiohead', plays='502'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='7ec864b4-660c-45d1-98af-bac9a8bf972d', artistname='anneke van giersbergen', plays='366'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='69158f97-4c07-4c4e-baf8-4e4ab1ed666e', artistname='boards of canada', plays='335'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='d0d33ef1-fcef-43c3-978e-9fe9e0156d7d', artistname='pogodno', plays='287'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='fc285e02-bb94-4ea9-ace8-aa1b625a1dc8', artistname='fisz emade jako tworzywo sztuczne', plays='243'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='aaf6e4a0-5009-4664-99d4-1b534277c686', artistname='smolik', plays='220'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='fc015f11-be38-4b7d-b3e8-5d809242ecec', artistname='clint mansell', plays='218'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='2119beb8-6ac5-4f21-82a4-b831c90c0024', artistname='damien rice', plays='217'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='7c158ea8-c0aa-410e-bdc1-20bba9759577', artistname='the cinematic orchestra', plays='206'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='b91f0973-002e-4237-a39f-c7e02e786e53', artistname='architecture in helsinki', plays='204'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='4ac4e32b-bd18-402e-adad-ae00e72f8d85', artistname='patrick wolf', plays='194'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='b10bbbfc-cf9e-42e0-be17-e2c3e1d2600d', artistname='the beatles', plays='187'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='f6f2326f-6b25-4170-b89d-e235b25508e8', artistname='sigur rós', plays='173'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='a1154343-1ea1-4007-97d3-2382482a90cd', artistname='stars of the lid', plays='172'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='e162b2eb-6a42-4240-8c1b-c94d9a0acb73', artistname='cocorosie', plays='156'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='b7723b3f-9a35-438d-bc42-8ad8b5e027ae', artistname='brad mehldau', plays='154'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid=None, artistname='marta dymek', plays='139'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d74786', artistid='36de1ab2-9ea8-405c-9cf4-d55d2ed55b80', artistname='marek napiórkowski', plays='136'),
 Row(userid='d139929ba9b232475a61b54e0421b942e8d7

In [0]:
# Creating dataframe from tsv
df = spark.read.csv('dbfs:/FileStore/tables/lastfm-data/usersha1-artmbid-artname-plays.tsv', sep='\t')
df.show(5)

+--------------------+--------------------+--------------------+----+
 _c0| _c1| _c2| _c3|
+--------------------+--------------------+--------------------+----+
00000c289a1829a80...|3bd73256-3905-4f3...| betty blowtorch|2137|
00000c289a1829a80...|f2fb0ff0-5679-42e...| die Ärzte|1099|
00000c289a1829a80...|b3ae82c2-e60b-455...| melissa etheridge| 897|
00000c289a1829a80...|3d6bbeb7-f90e-4d1...| elvenking| 717|
00000c289a1829a80...|bbd2ffd7-17f4-450...|juliette & the licks| 706|
+--------------------+--------------------+--------------------+----+
only showing top 5 rows

In [0]:
df = df.dropna()

In [0]:
newNames = ['userid', 'artistid', 'artistname', 'plays']
df = df.toDF(*newNames)
df.show(5)

+--------------------+--------------------+--------------------+-----+
 userid| artistid| artistname|plays|
+--------------------+--------------------+--------------------+-----+
00000c289a1829a80...|3bd73256-3905-4f3...| betty blowtorch| 2137|
00000c289a1829a80...|f2fb0ff0-5679-42e...| die Ärzte| 1099|
00000c289a1829a80...|b3ae82c2-e60b-455...| melissa etheridge| 897|
00000c289a1829a80...|3d6bbeb7-f90e-4d1...| elvenking| 717|
00000c289a1829a80...|bbd2ffd7-17f4-450...|juliette & the licks| 706|
+--------------------+--------------------+--------------------+-----+
only showing top 5 rows

In [0]:
# label encode users and artists
userIndexer = StringIndexer(inputCol="userid", outputCol="userIndex")
artistIndexer = StringIndexer(inputCol="artistid", outputCol="artistIndex")

userDecoder = IndexToString(inputCol="userIndex", outputCol="useridOrignial")
artistDecoder = IndexToString(inputCol="artistIndex", outputCol="artistidOriginal")

df = userIndexer.setHandleInvalid("keep").fit(df).transform(df) # sethandleInvalid("keep") because randomSplit was throwing error about string Indexer. Null values?
df = artistIndexer.setHandleInvalid("keep").fit(df).transform(df)

In [0]:
display(df.take(5))

userid,artistid,artistname,plays,userIndex,artistIndex
00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137,126839.0,17191.0
00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099,126839.0,277.0
00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897,126839.0,3120.0
00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717,126839.0,1981.0
00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706,126839.0,2121.0


In [0]:
df2 = df.withColumn("ratings", when(df.plays>50, 1).otherwise(0))
# df2.describe().show()

In [0]:
# data = df.select("userIndex", "artistIndex", "plays")
# data = df.selectExpr("INT(userIndex)","INT(artistIndex)","FLOAT(plays)","FLOAT(ratings)", "userid", "artistId")
data_demo = df2.selectExpr("INT(userIndex)","INT(artistIndex)","FLOAT(ratings)")

data_demo.dtypes

Out[9]: [('userIndex', 'int'), ('artistIndex', 'int'), ('ratings', 'float')]

In [0]:
(demo, rest) = data_demo.randomSplit([0.1, 0.9])

In [0]:
# Set the MLFLOW_TRACKING_URI environment variable to have MLflow find a URI from there. The URI can either be a HTTP/HTTPS URI for a remote server, a database connection string, or a local path to log data to a directory. 
mlflow.set_tracking_uri('databricks') 

In [0]:
# Set active Experiment. Creates new experiment if not available
mlflow.set_experiment(experiment_name ="/Users/sengarnh@mail.uc.edu/mxm/lastfm-als-experiment1")

In [0]:
mlflow.tracking.get_tracking_uri()

Out[48]: 'databricks'

In [0]:
# source: docs 
# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]), timestamp=int(p[3])))
# ratings = spark.createDataFrame(ratingsRDD)

alphas = [0.1, 0.5, 1]
ranks = [5, 10, 15]
for alpha in alphas:
  for rank in ranks:
    (training, test) = demo.randomSplit([0.8, 0.2])
    with mlflow.start_run(run_name='lastfm-als'):

      als = ALS(userCol="userIndex", itemCol="artistIndex", ratingCol="ratings",
                implicitPrefs=True)
      als.setMaxIter(5)
      als.setRegParam(0.1)
      als.setRank(rank)
      als.setAlpha(alpha)

      mlflow.log_param("model", "als")
      mlflow.log_param("maxIter", als.getMaxIter())
      mlflow.log_param("regParam", als.getRegParam())
      mlflow.log_param("rank", rank)
      mlflow.log_param("alpha", alpha)
      mlflow.log_param("implicitPrefs", als.getImplicitPrefs())

      model = als.fit(training)
      predictions = model.transform(test)
      predictions = predictions.filter(predictions.prediction != float('nan'))
      evaluator = RegressionEvaluator(metricName="rmse", labelCol="ratings",predictionCol="prediction")
      rmse = evaluator.evaluate(predictions)
      mlflow.log_metric("rmse", rmse)
      print("RANK= %s :Root-mean-square error = %s" %(str(rank), str(rmse)))

RANK= 5 :Root-mean-square error = 0.8190233774143151
RANK= 10 :Root-mean-square error = 0.8174932220055051
RANK= 15 :Root-mean-square error = 0.8180625027763793
RANK= 5 :Root-mean-square error = 0.8178814671322994
RANK= 10 :Root-mean-square error = 0.8181686483861481
RANK= 15 :Root-mean-square error = 0.8177828655636202
RANK= 5 :Root-mean-square error = 0.8171108113260362
RANK= 10 :Root-mean-square error = 0.8174162575761549
RANK= 15 :Root-mean-square error = 0.8153334980700492

In [0]:
run_id = mlflow.search_runs(filter_string='tags.mlflow.runName = "lastfm-als"').iloc[0].run_id

model_name = "lastfm-als-model"

model_version = mlflow.register_model(f"runs:/{run_id}/lastfm-als", model_name)

--------------------------------------------------------------------------- 
 RestException Traceback (most recent call last)
 <command-440421022018919> in <module> 
 3 model_name = "lastfm-als-model" 
 4 
 ----> 5 model_version = mlflow . register_model ( f"runs:/{run_id}/lastfm-als" , model_name ) 

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/_model_registry/fluent.py in register_model (model_uri, name, await_registration_for) 
 65 )
 66 else : 
 ---> 67 raise e
 68 
 69 if RunsArtifactRepository . is_runs_uri ( model_uri ) : 

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/_model_registry/fluent.py in register_model (model_uri, name, await_registration_for) 
 56 client = MlflowClient ( ) 
 57 try : 
 ---> 58 create_model_response = client . create_registered_model ( name ) 
 59 eprint ( "Successfully registered model '%s'." % create_model_response . name ) 
 60 except MlflowException as e : 

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/client.py in create_registered_model (self, name, tags, description) 
 1636 description : This sentiment analysis model classifies the tone - happy , sad , angry . 
 1637 """
 -> 1638 return self . _get_registry_client ( ) . create_registered_model ( name , tags , description ) 
 1639 
 1640 def rename_registered_model ( self , name : str , new_name : str ) -> RegisteredModel : 

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/_model_registry/client.py in create_registered_model (self, name, tags, description) 
 50 tags = tags if tags else { } 
 51 tags = [ RegisteredModelTag ( key , str ( value ) ) for key , value in tags . items ( ) ] 
 ---> 52 return self . store . create_registered_model ( name , tags , description ) 
 53 
 54 def update_registered_model ( self , name , description ) : 

 /databricks/python/lib/python3.8/site-packages/mlflow/store/model_registry/rest_store.py in create_registered_model (self, name, tags, description) 
 75 CreateRegisteredModel ( name = name , tags = proto_tags , description = description ) 
 76 )
 ---> 77 response_proto = self . _call_endpoint ( CreateRegisteredModel , req_body ) 
 78 return RegisteredModel . from_proto ( response_proto . registered_model ) 
 79 

 /databricks/python/lib/python3.8/site-packages/mlflow/store/model_registry/rest_store.py in _call_endpoint (self, api, json_body) 
 56 endpoint , method = _METHOD_TO_INFO [ api ] 
 57 response_proto = api . Response ( ) 
 ---> 58 return call_endpoint ( self . get_host_creds ( ) , endpoint , method , json_body , response_proto ) 
 59 
 60 # CRUD API for RegisteredModel objects 

 /databricks/python/lib/python3.8/site-packages/mlflow/utils/rest_utils.py in call_endpoint (host_creds, endpoint, method, json_body, response_proto) 
 168 host_creds = host_creds , endpoint = endpoint , method = method , json = json_body
 169 )
 --> 170 response = verify_rest_response ( response , endpoint ) 
 171 js_dict = json . loads ( response . text ) 
 172 parse_dict ( js_dict = js_dict , message = response_proto ) 

 /databricks/python/lib/python3.8/site-packages/mlflow/utils/rest_utils.py in verify_rest_response (response, endpoint) 
 120 if response . status_code != 200 : 
 121 if _can_parse_as_json ( response . text ) : 
 --> 122 raise RestException ( json . loads ( response . text ) ) 
 123 else : 
 124 base_msg = "API request to endpoint %s failed with error code " "%s != 200" % (

 RestException : PERMISSION_DENIED: Model Registry is not enabled for organization 6981538704621936.

In [0]:
if tracking_url_type_store != "file":
            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
  mlflow.log_model(model, "model", registered_model_name="ALS-Model")
else:
  mlflow.log_model(model, "model")

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-440421022018920> in <module> 
 4 # please refer to the doc for more information: 
 5 # https://mlflow.org/docs/latest/model-registry.html#api-workflow 
 ----> 6 mlflow . log_model ( model , "model" , registered_model_name = "ALS-Model" ) 
 7 else : 
 8 mlflow . log_model ( model , "model" ) 

 AttributeError : module 'mlflow' has no attribute 'log_model'

In [0]:
# # Generate top 10 movie recommendations for each user
# userRecs = model.recommendForAllUsers(10)
# # Generate top 10 user recommendations for each movie
# movieRecs = model.recommendForAllItems(10)

In [0]:
rmse = evaluator.evaluate(predictions)

In [0]:
rmse

Out[19]: 0.8153334980700492

In [0]:
model.save("./model/als-mlflow")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4142293834736427> in <module> 
 ----> 1 model . save ( "./model/als-mlflow" ) . overwrite ( ) 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 308 def save ( self , path ) : 
 309 """Save this ML instance to the given path, a shortcut of 'write().save(path)'.""" 
 --> 310 self . write ( ) . save ( path ) 
 311 
 312 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 259 if not isinstance ( path , str ) : 
 260 raise TypeError ( "path should be a string, got type %s" % type ( path ) ) 
 --> 261 self . _jwrite . save ( path ) 
 262 
 263 def overwrite ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o12849.save.
: java.io.IOException: Path ./model/als-mlflow already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
dbutils.fs.ls("./model/als-mlflow")

Out[27]: [FileInfo(path='dbfs:/model/als-mlflow/itemFactors/', name='itemFactors/', size=0),
 FileInfo(path='dbfs:/model/als-mlflow/metadata/', name='metadata/', size=0),
 FileInfo(path='dbfs:/model/als-mlflow/userFactors/', name='userFactors/', size=0)]

In [0]:
tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

In [0]:
tracking_url_type_store

Out[34]: ''

In [0]:
mlflow.get_tracking_uri()

Out[35]: 'databricks'